<div style="font-size: 200%; font-weight: bold; color: maroon;">DataFrames en SparkR</span></div>
<!-- v. 1.3 - Paulo Villegas, 2021 -->

Este notebook es una prueba con el objeto `SparkDataFrame` de [Spark R](https://spark.apache.org/docs/latest/sparkr.html)

# Inicialización

Carga de la librería y creadión de la `SparkSession`

In [ ]:
library(SparkR)
spark <- sparkR.session( "local[*]" );

# Lectura de datos

Vamos a usar [read.df](https://spark.apache.org/docs/latest/api/R/read.df.html) para leer un fichero [CSV](https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.sql.DataFrameReader) y convertirlo en un `SparkDataFrame`.

Trabajamos con la 
[Encuesta de inserción laboral de titulados universitarios](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176991&idp=1254735976597&menu=ultiDatos) (EILU) del año 2019

**Modalidad 1**: lectura "ciega"

In [ ]:
??read.df

In [ ]:
uni1 <- read.df("../DATA/situacion-laboral-titulados-2019.csv", source="csv", header=TRUE, comment="#")

Verificamos la clase del objeto

In [ ]:
class(uni1)

Y las columnas de la tabla

In [ ]:
names(uni1)

¿cuántas filas?

In [ ]:
count(uni1)

Nos traemos las 6 primeras filas

In [ ]:
head(uni1)

Veamos el esquema de la tabla

In [ ]:
printSchema(uni1)

**Modalidad 2**: le pedimos a Spark que deduzca el esquema

In [ ]:
uni2 <- read.df("../DATA/situacion-laboral-titulados-2019.csv", "csv", header=TRUE, 
                comment="#", inferSchema=TRUE)

In [ ]:
printSchema(uni2)

**Modalidad 3**: le especificamos nosotros el esquema

In [ ]:
schema.uni <- structType(structField("Rama", "string"), 
                     structField("TipoUniversidad", "string"),
                     structField("Sexo","string"),
                     structField("Total","integer"),
                     structField("Trabajando","integer"),
                     structField("Desempleo","integer"),
                     structField("Inactivo","integer")
                    )

In [ ]:
uni3 <- read.df( "../DATA/situacion-laboral-titulados-2019.csv", "csv", header=TRUE, 
                comment="#", schema=schema.uni )

In [ ]:
printSchema(uni3)

In [ ]:
head(uni3)

# Manipulación de datos

## Agrupaciones
¿Cuántos licenciados hubo en 2013-2014, por sexos?

In [ ]:
totales.por.sexos <- summarize( groupBy(uni2,"Sexo"), num=sum(uni2$Total))

In [ ]:
collect(totales.por.sexos)

## Filtrado de filas

Con cláusulas explícitas

In [ ]:
uni.f = filter( uni2, uni2$tipo == "públicas")

In [ ]:
uni.f = filter( uni2, (uni2$Sexo == "mujeres") & (lower(uni2$Rama) == "ciencias") )

Con cláusulas dentro de cadenas (tipo SQL)

In [ ]:
uni.f = filter( uni2, 'Sexo = "hombre" AND Rama = "Ciencias"' )

In [ ]:
uni.f = filter( uni2, 'Sexo = "hombre" AND Rama LIKE "Ciencias%"' )

In [ ]:
head(uni.f)

## Selección de columnas


In [ ]:
uni.trabajando = select( uni2, "Rama", "Total", "Trabajando")

In [ ]:
head( uni.trabajando )

## Ordenación

In [ ]:
uni.o = arrange( uni.trabajando, "Trabajando", decreasing=TRUE)

In [ ]:
head(uni.o)

## Creación de columnas
Cálculo de la fracción de personas empleadas en cada rama 

In [ ]:
uni.fraction = mutate( uni.trabajando, frac=uni.trabajando$Trabajando/uni.trabajando$Total)

In [ ]:
# Veamos una muestra
head( uni.fraction )

Vamos a representar gráficamente 
(<span style='color: red; font-style: italic;'>¡esto solo se puede hacer si el tamaño de lo que vamos a recuperar lo permite!</span>)

In [ ]:
library(ggplot2)

In [ ]:
# Pintemos el resultado
options(repr.plot.width = 10, repr.plot.height = 5)
qplot( Rama, frac, data=collect(uni.fraction) )

recuperemos el resultado para examinarlo

In [ ]:
collect(uni.fraction)

Si queremos saber la fracción _real_ de personas trabajando por rama, necesitamos agrupar ...

In [ ]:
uni.tot = summarize( groupBy(uni.trabajando, uni.trabajando$Rama), 
                     Total=sum(uni.trabajando$Total),
                     Trabajando=sum(uni.trabajando$Trabajando) )

In [ ]:
collect(uni.tot)

In [ ]:
uni.fraction = mutate( uni.tot, frac=uni.tot$Trabajando/uni.tot$Total)

In [ ]:
collect(uni.fraction)

In [ ]:
qplot( Rama, frac, data=collect(uni.fraction) ) + geom_bar(stat="identity")

# Piping

El uso de la librería `magrittr`, al estilo de `dplyr`, puede producir código más claro mediante el encadenamiento de operaciones usando `%>%`. Aunque los verbos de SparkR a veces tienen problemas con algunos parámetros, cuando no son cadenas.

In [ ]:
library( 'magrittr' )

In [ ]:
uni.f  <- uni2 %>%
    filter( 'Sexo = "mujeres" AND Rama LIKE "Ciencias%"' ) %>%
    arrange( 'Desempleo', decreasing=TRUE ) %>%
    collect()

uni.f

# Consultas SQL
Podemos registrar un dataframe como tabla temporal en el sistema, y realizar consultas sobre ella

In [ ]:
createOrReplaceTempView( uni2, "unitable")

In [ ]:
r = sql( "SELECT * from unitable WHERE Total > 10000")

In [ ]:
collect( r )

# Escritura de tablas

También podemos guardar un SparkDataFrame como "tabla de sistema" permanente usando [saveAsTable](https://spark.apache.org/docs/latest/api/R/saveAsTable.html)

El destino exacto de la tabla de "sistema" dependerá de cómo esté configurado Spark (y del parámetro opcional `source`). Puede ser por ejemplo como fichero Parquet en el sistema de ficheros soportado por el cluster de Spark, o como tabla en un despliegue de Hive en el cluster donde está desplegado Spark.

En cualquier caso, el efecto neto es que el SparkDataFrame queda almacenado de forma persistente, es decir, aunque el proceso _driver_ de Spark que creó la tabla termine, los datos no desaparecen (cosa que sí que pasa con un `SparkDataFrame`, o si la tabla se crea mediante `createOrReplaceTempView`), y procesos posteriores pueden recuperarlos.

In [ ]:
saveAsTable( uni2, "situacion_laboral_titulados_2019", mode="overwrite")

Es posible obtener la lista de tablas almacenadas en el sistema:

In [ ]:
table.list = tables()

In [ ]:
collect(table.list)

In [ ]:
# También podemos borrar la tabla
sql("DROP TABLE situacion_laboral_titulados_2019")

# Ejercicios

## Comparación entre tipos de universidad

<span style="color: red;">Pregunta:</span> *El ratio de empleados por rama temática calculado antes, ¿varía en función del tipo de universidad?. Es decir, para cada rama temática, ¿la diferencia entre universidad pública y privada es siempre similar o cambia mucho?*

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Escriba el código necesario en la siguiente celda</div>

## Ratios de desempleados por tipo de universidad y sexo

<span style="color: red;">Pregunta:</span> *el ratio de desempleados/total en la universidad pública ¿es muy diferente para hombres y mujeres?*

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Escriba el código necesario en la siguiente celda</div>